In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [19]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [20]:
demo = pd.read_csv("PTDEMOG.csv")

In [21]:
neuro = pd.read_csv("NEUROEXM.csv")

In [22]:
neuro.columns

Index(['Phase', 'ID', 'RID', 'SITEID', 'VISCODE', 'VISCODE2', 'USERDATE',
       'USERDATE2', 'EXAMDATE', 'NXVISUAL', 'NXVISDES', 'NXAUDITO', 'NXAUDDES',
       'NXTREMOR', 'NXTREDES', 'NXCONSCI', 'NXCONDES', 'NXNERVE', 'NXNERDES',
       'NXMOTOR', 'NXMOTDES', 'NXFINGER', 'NXFINDES', 'NXHEEL', 'NXHEEDES',
       'NXSENSOR', 'NXSENDES', 'NXTENDON', 'NXTENDES', 'NXPLANTA', 'NXPLADES',
       'NXGAIT', 'NXGAIDES', 'NXOTHER', 'NXOTHSPE', 'NXGENCOM', 'NXABNORM',
       'update_stamp'],
      dtype='object')

In [23]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [24]:
clinical.head()

,RID,SUBJID,Phase,VISCODE,VISCODE2,EXAMDATE,PHC_VISIT,PHC_Sex,PHC_Education,PHC_Ethnicity,PHC_Race,PHC_AGE,PHC_Diagnosis,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,5.0,74.439425,1.0,0.277,0.254,0.293,NaN,2022-10-18 09:15:45.0
1,2,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,1.0,16.0,2.0,5.0,74.929500,1.0,0.237,0.278,0.535,-0.333,2022-10-18 09:15:45.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,1.0,16.0,2.0,5.0,77.407255,1.0,0.344,0.419,0.535,NaN,2022-10-18 09:15:45.0
3,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,1.0,16.0,2.0,5.0,79.477070,1.0,0.101,0.066,0.620,0.264,2022-10-18 09:15:45.0
4,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,1.0,16.0,2.0,5.0,80.468172,1.0,0.090,0.002,0.460,0.264,2022-10-18 09:15:45.0


In [25]:
clinical = clinical.rename(columns = {"PHC_Diagnosis":"DX"})

In [26]:
diag["Subject"].value_counts()

022_S_0096    4
123_S_0298    4
027_S_0074    4
002_S_1280    4
021_S_0337    4
             ..
094_S_1397    1
024_S_1400    1
094_S_1402    1
082_S_0641    1
016_S_6836    1
Name: Subject, Length: 2838, dtype: int64

In [27]:
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "Phase"]]

In [28]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [29]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [30]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [31]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [32]:
m["PTID"].value_counts()

067_S_4767    18
141_S_0697    15
131_S_0384    15
137_S_0973    14
126_S_1077    14
              ..
094_S_5155     1
127_S_5201     1
041_S_5204     1
073_S_5215     1
131_S_6153     1
Name: PTID, Length: 2453, dtype: int64

In [33]:
t = m

In [34]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE", "PTDOBMM","DX"], axis=1) 

In [35]:
t.columns

Index(['RID', 'PTID', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND', 'PTMARRY',
       'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTWRECNT', 'PTNOTRT', 'PTRTYR',
       'PTHOME', 'PTOTHOME', 'PTTLANG', 'PTPLANG', 'PTPSPEC', 'PTCOGBEG',
       'PTMCIBEG', 'PTADBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'EXAMDATE',
       'NXVISUAL', 'NXVISDES', 'NXAUDITO', 'NXAUDDES', 'NXTREMOR', 'NXTREDES',
       'NXCONSCI', 'NXCONDES', 'NXNERVE', 'NXNERDES', 'NXMOTOR', 'NXMOTDES',
       'NXFINGER', 'NXFINDES', 'NXHEEL', 'NXHEEDES', 'NXSENSOR', 'NXSENDES',
       'NXTENDON', 'NXTENDES', 'NXPLANTA', 'NXPLADES', 'NXGAIT', 'NXGAIDES',
       'NXOTHER', 'NXOTHSPE', 'NXGENCOM', 'NXABNORM', 'SUBJID', 'PHC_VISIT',
       'PHC_Sex', 'PHC_Education', 'PHC_Ethnicity', 'PHC_Race', 'PHC_AGE',
       'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [36]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [37]:
len(t.columns)

46

In [38]:
t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
                                                                                                                   "bookkeeper").str.replace("cpa", "accounting")

In [39]:
t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

In [40]:
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

C:\Users\HP\AppData\Local\Temp\ipykernel_25272\2309654543.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
C:\Users\HP\AppData\Local\Temp\ipykernel_25272\2309654543.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
C:\Users\HP\AppData\Local\Temp\ipykernel_25272\2309654543.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
C:\Users\HP\AppData\Local\Temp\ipykernel_25272\2309654543.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
C:\Users\HP\AppData\Local\Temp\ipykernel_25272\2309654543.py:5: FutureWarning: T

In [41]:
cond = t['PTWORK'].value_counts()
threshold = 10
t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [42]:
categorical = ['PTGENDER', 'PTWORK',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [43]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [44]:
text = ["PTWORK", "CMMED"]

In [45]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

,PHC_Education,EXAMDATE,PTRTYR,PHC_MEM,NXOTHER,PHC_Ethnicity,PTETHCAT,NXCONSCI,PTADDX,PHC_AGE,...,PHC_EXF,SUBJID,PHC_Race,PTRACCAT,NXSENSOR,NXABNORM,NXPLANTA,PTTLANG,PTWRECNT,PHC_VISIT
0,18.0,2005-08-18,9/18/1989,-1.068,1.0,2.0,2.0,1.0,-4.0,81.366188,...,-0.670,G-ADNI-AN000439,5.0,5.0,1.0,1.0,1.0,1.0,chemical engineer,1
1,18.0,2005-08-18,9/18/1989,-0.768,1.0,2.0,2.0,1.0,-4.0,81.864476,...,-0.145,G-ADNI-AN000439,5.0,5.0,1.0,1.0,1.0,1.0,chemical engineer,2
2,18.0,2005-08-18,9/18/1989,-0.973,1.0,2.0,2.0,1.0,-4.0,82.365503,...,-1.036,G-ADNI-AN000439,5.0,5.0,1.0,1.0,1.0,1.0,chemical engineer,3
3,18.0,2005-08-18,9/18/1989,-1.559,1.0,2.0,2.0,1.0,-4.0,83.364819,...,-0.645,G-ADNI-AN000439,5.0,5.0,1.0,1.0,1.0,1.0,chemical engineer,4
4,10.0,2005-08-18,-4,-0.014,1.0,1.0,1.0,1.0,-4.0,67.852156,...,-0.603,G-ADNI-AN000446,5.0,5.0,1.0,1.0,1.0,2.0,Mechanic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,20.0,-4,2009-xx-xx,0.624,1.0,2.0,2.0,1.0,9999.0,80.980151,...,-4.000,-4,5.0,5.0,1.0,1.0,1.0,1.0,Anesthesiologist,1
7976,18.0,-4,2009-xx-xx,-0.180,-4.0,2.0,2.0,1.0,9999.0,76.599589,...,-4.000,-4,5.0,5.0,1.0,1.0,1.0,1.0,Economist state government,1
7977,14.0,-4,2019-02-xx,0.870,-4.0,2.0,2.0,-4.0,-4.0,74.759754,...,-4.000,-4,5.0,5.0,-4.0,2.0,-4.0,1.0,conpanion,1
7978,16.0,-4,2007-11-30,-0.814,2.0,2.0,2.0,1.0,2019.0,74.680356,...,-4.000,-4,5.0,5.0,1.0,1.0,1.0,1.0,real estate broker,1


In [46]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXOTHER
PHC_Ethnicity
PTETHCAT
NXCONSCI
NXFINGER
PHC_Sex
NXGAIT
NXMOTOR
NXNERVE
NXHEEL
NXTENDON
PHC_Race
PTRACCAT
NXSENSOR
NXABNORM
NXPLANTA
PTTLANG
PHC_VISIT


In [48]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJID", "PTWRECNT"]
t = t.drop(to_del, axis=1)

In [49]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

,PTDOBYY,PHC_Education,PTADDX,PHC_LAN,PHC_AGE,PHC_MEM,PTCOGBEG,PHC_EXF,PHC_VSP
0,1924.0,18.0,-4.0,-0.516,81.366188,-1.068,-4.0,-0.670,-0.237
1,1924.0,18.0,-4.0,-0.516,81.864476,-0.768,-4.0,-0.145,-0.305
2,1924.0,18.0,-4.0,-0.828,82.365503,-0.973,-4.0,-1.036,0.128
3,1924.0,18.0,-4.0,-0.756,83.364819,-1.559,-4.0,-0.645,-4.000
4,1938.0,10.0,-4.0,0.487,67.852156,-0.014,-4.0,-0.603,-0.074
...,...,...,...,...,...,...,...,...,...
7975,1938.0,20.0,9999.0,-4.000,80.980151,0.624,2017.0,-4.000,-4.000
7976,1943.0,18.0,9999.0,-4.000,76.599589,-0.180,2009.0,-4.000,-4.000
7977,1945.0,14.0,-4.0,-4.000,74.759754,0.870,2014.0,-4.000,-4.000
7978,1945.0,16.0,2019.0,-4.000,74.680356,-0.814,2019.0,-4.000,-4.000


In [50]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [51]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [52]:
t[l]

,RID,PTID,Group,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTWORK,...,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.068,-0.670,-0.516,-0.237
1,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.768,-0.145,-0.516,-0.305
2,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.973,-1.036,-0.828,0.128
3,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,engineer,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.559,-0.645,-0.756,-4.000
4,4,022_S_0004,1.0,ADNI1,1.0,1938.0,1.0,1.0,10.0,other,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.014,-0.603,0.487,-0.074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,6725,013_S_6725,1.0,ADNI3,1.0,1938.0,1.0,3.0,20.0,other,...,1.0,1.0,1.0,1.0,1.0,1.0,0.624,-4.000,-4.000,-4.000
7976,6851,168_S_6851,1.0,ADNI3,2.0,1943.0,1.0,3.0,18.0,nurse,...,1.0,1.0,2.0,1.0,2.0,1.0,-0.180,-4.000,-4.000,-4.000
7977,6841,035_S_6841,0.0,ADNI3,2.0,1945.0,1.0,1.0,14.0,sales,...,-4.0,-4.0,-4.0,-4.0,-4.0,2.0,0.870,-4.000,-4.000,-4.000
7978,6855,032_S_6855,2.0,ADNI3,1.0,1945.0,1.0,1.0,16.0,analyst,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.814,-4.000,-4.000,-4.000


In [53]:
dfs = []

In [54]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [55]:
cat = pd.concat(dfs, axis=1)

In [56]:
t[quant]

,PTDOBYY,PHC_Education,PTADDX,PHC_LAN,PHC_AGE,PHC_MEM,PTCOGBEG,PHC_EXF,PHC_VSP
0,1924.0,18.0,-4.0,-0.516,81.366188,-1.068,-4.0,-0.670,-0.237
1,1924.0,18.0,-4.0,-0.516,81.864476,-0.768,-4.0,-0.145,-0.305
2,1924.0,18.0,-4.0,-0.828,82.365503,-0.973,-4.0,-1.036,0.128
3,1924.0,18.0,-4.0,-0.756,83.364819,-1.559,-4.0,-0.645,-4.000
4,1938.0,10.0,-4.0,0.487,67.852156,-0.014,-4.0,-0.603,-0.074
...,...,...,...,...,...,...,...,...,...
7975,1938.0,20.0,9999.0,-4.000,80.980151,0.624,2017.0,-4.000,-4.000
7976,1943.0,18.0,9999.0,-4.000,76.599589,-0.180,2009.0,-4.000,-4.000
7977,1945.0,14.0,-4.0,-4.000,74.759754,0.870,2014.0,-4.000,-4.000
7978,1945.0,16.0,2019.0,-4.000,74.680356,-0.814,2019.0,-4.000,-4.000


In [57]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTWORK_accounting,PTWORK_admin,PTWORK_analyst,PTWORK_architect,PTWORK_army officer,PTWORK_artist,...,PTTLANG_2.0,PHC_VISIT_1,PHC_VISIT_2,PHC_VISIT_3,PHC_VISIT_4,PHC_VISIT_5,PHC_VISIT_6,PHC_VISIT_7,PHC_VISIT_8,PHC_VISIT_9
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7976,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7977,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7978,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [58]:
t[["PTID","RID", "Phase", "Group"]]

,PTID,RID,Phase,Group
0,011_S_0003,3,ADNI1,2.0
1,011_S_0003,3,ADNI1,2.0
2,011_S_0003,3,ADNI1,2.0
3,011_S_0003,3,ADNI1,2.0
4,022_S_0004,4,ADNI1,1.0
...,...,...,...,...
7975,013_S_6725,6725,ADNI3,1.0
7976,168_S_6851,6851,ADNI3,1.0
7977,035_S_6841,6841,ADNI3,0.0
7978,032_S_6855,6855,ADNI3,2.0


In [59]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [60]:
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTWORK_accounting,PTWORK_admin,...,PHC_VISIT_9,PTDOBYY,PHC_Education,PTADDX,PHC_LAN,PHC_AGE,PHC_MEM,PTCOGBEG,PHC_EXF,PHC_VSP
0,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,1924.0,18.0,-4.0,-0.516,81.366188,-1.068,-4.0,-0.670,-0.237
1,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,1924.0,18.0,-4.0,-0.516,81.864476,-0.768,-4.0,-0.145,-0.305
2,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,1924.0,18.0,-4.0,-0.828,82.365503,-0.973,-4.0,-1.036,0.128
3,011_S_0003,3,ADNI1,2.0,0,1,0,0,0,0,...,0,1924.0,18.0,-4.0,-0.756,83.364819,-1.559,-4.0,-0.645,-4.000
4,022_S_0004,4,ADNI1,1.0,0,1,0,0,0,0,...,0,1938.0,10.0,-4.0,0.487,67.852156,-0.014,-4.0,-0.603,-0.074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,013_S_6725,6725,ADNI3,1.0,0,1,0,0,0,0,...,0,1938.0,20.0,9999.0,-4.000,80.980151,0.624,2017.0,-4.000,-4.000
7976,168_S_6851,6851,ADNI3,1.0,0,0,1,0,0,0,...,0,1943.0,18.0,9999.0,-4.000,76.599589,-0.180,2009.0,-4.000,-4.000
7977,035_S_6841,6841,ADNI3,0.0,0,0,1,0,0,0,...,0,1945.0,14.0,-4.0,-4.000,74.759754,0.870,2014.0,-4.000,-4.000
7978,032_S_6855,6855,ADNI3,2.0,0,1,0,0,0,0,...,0,1945.0,16.0,2019.0,-4.000,74.680356,-0.814,2019.0,-4.000,-4.000


In [61]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [62]:
c.to_csv("clinical.csv")

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.to_pickle("X_train_c.pkl")
y_train.to_pickle("y_train_c.pkl")

X_test.to_pickle("X_test_c.pkl")
y_test.to_pickle("y_test_c.pkl")